In [1]:
import os
import numpy as np
import pandas as pd
import itertools

## Read Data

In [17]:
def read_data(location):
    """
    This function reads the data from the excel file.
    The excel file should have two tabs named below:
    1. Items
    2. Transactions
    The function reads the data without the header and assigns header to the dataframe afterwards.
    To run the program with a new set of data, please create an .xlsx file with two above mentioned tabs and please do not
    put header in the data as teh function automatically assigns header to avoid errors and naming conventions
    Returns :
    1. Pandas DataFrame (columns :["Item#","ItemName"])
    1. Pandas DataFrame (columns :  ["Transaction ID","Transaction"])
    """
    items = pd.read_excel(location, sheet_name = "Items", header=None,names = ["Item#","ItemName"])
    # print (items.columns)
    transactions = pd.read_excel(location, sheet_name = "Transactions", header=None,names = ["Transaction ID","Transaction"])
    # print (transactions.columns)
    print ("total number of distinct items : ",len(items.index))
    print ("total number of transactions : ", len(transactions.index))
    return items,transactions

## Calculate Support

In [18]:
def calculate_support(transaction,item):
    """
    This function reads the data from the excel file.
    The excel file should have two tabs named below:
    1. Items
    2. Transactions
    The function reads the data without the header and assigns header to the dataframe afterwards.
    To run the program with a new set of data, please create an .xlsx file with two above mentioned tabs and please do not
    put header in the data as teh function automatically assigns header to avoid errors and naming conventions
    Returns :
    1. Pandas DataFrame (columns :["Item#","ItemName"])
    1. Pandas DataFrame (columns :  ["Transaction ID","Transaction"])
    """
    Total_transactions = len(transaction.index)
    item_temp = item
    item_temp["Support"] = 0.0
    item_temp.Support = item_temp.Support.astype(float)
    for i in item_temp.index:
        item_list = item_temp.ItemName[i].split(",")
        # print (len(item_list),item_list)
        supp = 0
        for j in transaction.index:
            # print (transaction.Transaction[j].split(","))
            if all(x in map(str.strip,transaction.Transaction[j].split(",")) for x in item_list):
                supp = supp + 1
        item_temp.loc[i,"Support"] = float(supp)/Total_transactions
    return item_temp.sort_values(by='Support',ascending=False).reset_index(drop=True)

## Exclude by minimum support (min_support : User Input)

In [19]:
def exclusion_process(items,transactions,min_support):
    """
    :param items: Pandas DataFrame (columns :["Item#","ItemName"]) | Source function : read_data()
    :param transactions: Pandas DataFrame (columns :  ["Transaction ID","Transaction"]) | Source function : read_data()
    :param min_support: Minimum Support ( Provided by the User while running )
    :return: Two Pandas DataFrame.
             1. low_freq_combi --> Combinations of Items which has Support below min_support
             2. high_frequent_combi --> Combinations of Items which has Support above min_support
    """
    k = 1
    low_freq_combi = []
    high_frequent_combi = pd.DataFrame(columns=["ItemName","Support"])
    distinct_items = list(set(items.ItemName))
    # print (distinct_items)

    while k < len(distinct_items):
        k_freq_df = pd.DataFrame(columns=["ItemName","Support"])
        k_freq_df["ItemName"] = [",".join(i) for i in itertools.combinations(list(items.ItemName), k)]
        support_df = calculate_support(transactions, k_freq_df)
        # print (support_df)
        low_freq_df = support_df[support_df.Support < min_support]
        low_freq_df = low_freq_df.reset_index()
        for lf in low_freq_df.index:
            low_freq_combi.append(low_freq_df.ItemName[lf])

        support_df = support_df[support_df.Support >= min_support]

        for s in support_df.index:
            l = len(high_frequent_combi.index)
            high_frequent_combi.loc[l, "ItemName"] = support_df.ItemName[s]
            high_frequent_combi.loc[l, "Support"] = support_df.Support[s]
        k = k+1
    return low_freq_combi,high_frequent_combi

## Pruning

In [20]:
def pruning(low_freq_combination,high_frequent_combination):
    """
    :param low_freq_combination: Pandas DataFrame ( source : exclusion_process() )
    :param high_frequent_combination: Pandas DataFrame ( source : exclusion_process() )
    :return: Pandas DataFrame returns Pruned High Frequent Combination on the basis of
            "superset of low Support items
             would also have low Support"
    """
    for i in low_freq_combination:
        i = i.split(",")
        for j in high_frequent_combination.index:
            if all(x in map(str.strip, high_frequent_combination.ItemName[j].split(",")) for x in i):
                high_frequent_combination = high_frequent_combination.drop(j)
                high_frequent_combination = high_frequent_combination.reset_index(drop=True)
    return high_frequent_combination

## Calculate Confidence

In [21]:
def calculate_confidence(A,B,support_df):
    """
    This function calculates the confidence of any given two sets with their support
                    Confidence(X,Y) = Support(X,Y)/Support(X)
    :return:
             printable_A --> Returns a string. Returns a string with '&' separator If there is multiple items
             printable_B --> Returns a string. Returns a string with '&' separator If there is multiple items
             support_ab --> Support of A,B
             confidence --> Calculated confidence
    """
    printable_A = " & ".join(str(x) for x in sorted(list(A)))
    printable_B = " & ".join(str(x) for x in sorted(list(B)))
    A_str = ",".join(str(x) for x in sorted(list(A)))
    AB_str = ",".join(str(x) for x in sorted(list(A) + list(B)))
    support_df["ItemName"] = [",".join(y for y in sorted(list(x))) for x in support_df.item_list]
    '''
    print (support_df)
    print (A_str)
    print (B_str)
    '''
    support_a = support_df[support_df.ItemName == A_str].Support.values[0]
    support_ab = support_df[support_df.ItemName == AB_str].Support.values[0]

    confidence = float(support_ab) / support_a
    return printable_A,printable_B,support_ab,confidence

## Generate & Save Association Rules

In [23]:
def generate_association_rules(pruned_frequent_items, min_confidence):
    """
    :param pruned_frequent_items: Pandas DataFrame of pruned items who has support more than min_support (source : pruning())
    :return: Pandas DataFrame with all the Rules,Support and Confidence for all the possible permutations
    """
    Association_Rules = pd.DataFrame(columns=["Association_Rules","Support(%)","Confidence(%)"])
    pruned_frequent_items["item_list"] = [x.split(",") for x in pruned_frequent_items["ItemName"]]
    pruned_frequent_items["item_length"] = [len(x) for x in pruned_frequent_items["item_list"]]
    eligible_combinations = pruned_frequent_items[pruned_frequent_items.item_length > 1]
    # print (eligible_combinations)

    for i in eligible_combinations.index:
        for p in list(itertools.permutations(eligible_combinations.item_list[i])):
            for n in range(1,len(p)):
                rule_str_left,rule_str_right,support,confidence = calculate_confidence(list(p[:-n]), p[-n:], pruned_frequent_items)
                # print (rule_str_left,"-->",rule_str_right," | Support: ",support * 100,"%"," Confidence: ",confidence * 100, "%")
                Association_Rules.loc[-1] = [rule_str_left + " --> " + rule_str_right,support*100,confidence*100]
                Association_Rules.index = Association_Rules.index + 1
                Association_Rules = Association_Rules.sort_index()
    Association_Rules = Association_Rules.drop_duplicates()
    Association_Rules = Association_Rules.reset_index(drop = True)
    Association_Rules = Association_Rules.sort_index(ascending=False)
    Association_Rules = Association_Rules.reset_index(drop = True)
    Association_Rules = Association_Rules[Association_Rules["Confidence(%)"] >= min_confidence * 100]
    Association_Rules.to_csv(os.getcwd() + "/output/Association_Rules_")
    return Association_Rules

## Run Apriori (Wrapper Function)

In [25]:
def run_Apriori(items, transactions, min_support, min_cofidence):
    low_freq_combi, high_frequent_combi = exclusion_process(items, transactions, min_support)
    # print (high_frequent_combi)
    high_frequent_combi = pruning(low_freq_combi, high_frequent_combi)
    association_rules = generate_association_rules(high_frequent_combi,min_cofidence)
    #print (association_rules)
    return association_rules.head(len(association_rules.index))

## Please Run the below code block everytime to run Apriori Algorithm and Generate Rules for Any given Dataset and Parameters

In [67]:
input_location = os.getcwd() + "/input/"
all_files_available = [f for f in os.listdir(input_location) if os.path.isfile(os.path.join(input_location, f))]
all_xlsx_inputs_available = [x for x in all_files_available if x[-5:] == '.xlsx']
print ("\n-------------------PARAMETERS-------------------\n")
location = input_location + str(input("Enter the Database Name from the following list \n" + ''.join(['--'+x+'\n' for x in all_xlsx_inputs_available]) + "\nFile Name: "))
min_support = float(input("Enter a minimum support value: "))
min_confidence = float(input("Enter a minimum confidence value: "))
print ("\n-------------------------------------------------\n")
print ("\n-------------------Association Rules-------------------\n")
items, transactions = read_data(location)
run_Apriori(items, transactions, min_support, min_confidence)


-------------------PARAMETERS-------------------

Enter the Database Name from the following list 
--DATABASE_1_AMAZON.xlsx
--DATABASE_4_NIKE.xlsx
--Modified_Generic_Data_Canvas.xlsx
--Generic_Data_Canvas.xlsx
--DATABASE_3_KMART.xlsx
--Homework_1_Apriori.xlsx
--DATABASE_2_BESTBUY.xlsx

File Name: DATABASE_1_AMAZON.xlsx
Enter a minimum support value: 0.50
Enter a minimum confidence value: 0.70

-------------------------------------------------


-------------------Association Rules-------------------

total number of distinct items :  10
total number of transactions :  20


,Association_Rules,Support(%),Confidence(%)
0,Java: The Complete Reference --> Java For Dummies,50.0,100.000000
1,Java For Dummies --> Java: The Complete Reference,50.0,76.923077
